# Weight of Evidence

In [ ]:
# Get number of events and non-events per sub-category in a categorical variable
a = pd.DataFrame(pd.crosstab(df['Col1'],df['Event_Occurred/Not'])) 
# Get the top 6 sub categories having the most no. of events
a = a.sort_values(by=1, ascending=False)
#a.head(13)
b = a[:6]

# Find Weight of evidence & Information Value
b.iloc[5,0] = a[0].iloc[5:].sum()
b.iloc[5,1] = a[1].iloc[5:].sum()
b['Event_A'] = b[1]
b['Non-Event_A'] = b[0]
del b[0]
del b[1]
b['%Event']= b['Event_A']/b['Event_A'].sum()
b['%Non-Event']= b['Non-Event_A']/b['Non-Event_A'].sum()
b['%Non-Event'] = b['%Non-Event'].astype(float)
b['%Event'] = b['%Event'].astype(float)
import math
b['WOE']=np.log(b['%Non-Event']/b['%Event'])
b['%Event - %Non-Event']=b['%Event'] - b['%Non-Event']
b['IV'] = abs(b['%Event - %Non-Event']*b['WOE'])
b

# KS-Score for Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20,random_state=0)

clf = RandomForestClassifier(n_estimators=36,max_depth=3, max_features='sqrt', random_state=10) # max_features: reduces the variation in accuracy for same data, random_state: reduces variation in accuracy for different train-test data
clf.fit(X_train,Y_train)
Y_pred=clf.predict(X_test)

predictions = clf.predict_proba(X_test) # To get predicted probabilities
predictions=pd.DataFrame(predictions)
predictions.columns=["p0","p1"]
p=predictions['p1']
p=pd.DataFrame(p,columns=['p1'])
p.columns=['p1']
Y_test=pd.DataFrame(Y_test)
df=pd.concat([Y_test.reset_index(drop=True),p.reset_index(drop=True)],axis=1)
df

########################################################################################

#KS - Score in the form of deciles (having equal size)
a = list(range(1,11))
b = list(np.arange(0,1,0.1))
b = list(np.around(np.array(b),2))
c = list(np.arange(0.1,1.1,0.1))
c = list(np.around(np.array(c),2))
data ={'Decile':a,
       'Min-Prob': b,
       'Max-Prob':c}
t1 = pd.DataFrame(data)

data1 = df[df['INC_SSO Occur'] ==1]
data0 = df[df['INC_SSO Occur'] ==0]
t1['Event'] = 0
t1['Non-Event'] = 0
for i in range(0,len(t1)):       # use range instead of directly accessing the data, otherwise for loop skips data
    minp=t1['Min-Prob'].iloc[i]
    maxp=t1['Max-Prob'].iloc[i]
    e = (data1['p1'] > minp) & (data1['p1'] <= maxp)
    ne = (data0['p1'] > minp) & (data0['p1'] <= maxp)
    t1.loc[(t1['Min-Prob'] == minp) & (t1['Max-Prob'] == maxp), 'Event'] = e.values.sum()
    t1.loc[(t1['Min-Prob'] == minp) & (t1['Max-Prob'] == maxp), 'Non-Event'] = ne.values.sum()
    
t1['Event Rate'] = t1['Event'] / t1['Event'].sum()
t1['Non-Event Rate'] = t1['Non-Event'] / t1['Non-Event'].sum()
t1['% Cum Event Rate'] = (t1['Event Rate'].cumsum())*100
t1['% Cum Non-Event Rate'] = (t1['Non-Event Rate'].cumsum())*100
t1['KS'] = round((t1['% Cum Non-Event Rate'] - t1['% Cum Event Rate']),2)
print('KS Score: ' + str(round(max(t1['KS']),2)) + '%')
t1